# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
airbnb_price = spark.read.option("header", "true").csv("/FileStore/Notebook Examen/Caso3/airbnb_price.csv", sep=",")
airbnb_room_type = spark.read.option("header", "true").csv("/FileStore/Notebook Examen/Caso3/airbnb_room_type.csv")
airbnb_last_review = spark.read.option("header", "true").csv("/FileStore/Notebook Examen/Caso3/airbnb_last_review.tsv", sep="\t")

print("Airbnb price \n" , airbnb_price.head())
print("airbnb room type \n" , airbnb_room_type.head())
print("airbnb last review \n" , airbnb_last_review.head())

Airbnb price 
 Row(listing_id='2595', price='225 dollars', nbhood_full='Manhattan, Midtown')
airbnb room type 
 Row(listing_id='2595', description='Skylit Midtown Castle', room_type='Entire home/apt')
airbnb last review 
 Row(listing_id='2595', host_name='Jennifer', last_review='May 21 2019')


In [0]:
#fusionamos los Datframes
airbnb_listings = airbnb_price.join(airbnb_room_type, "listing_id").join(airbnb_last_review, "listing_id")

In [0]:
#Mostramos el Dataframe combinado
print("Airbnb Listings:")
airbnb_listings.show(5)

Airbnb Listings:
+----------+-----------+--------------------+--------------------+---------------+-----------+------------+
|listing_id|      price|         nbhood_full|         description|      room_type|  host_name| last_review|
+----------+-----------+--------------------+--------------------+---------------+-----------+------------+
|      2595|225 dollars|  Manhattan, Midtown|Skylit Midtown Ca...|Entire home/apt|   Jennifer| May 21 2019|
|      3831| 89 dollars|Brooklyn, Clinton...|Cozy Entire Floor...|Entire home/apt|LisaRoxanne|July 05 2019|
|      5099|200 dollars|Manhattan, Murray...|Large Cozy 1 BR A...|Entire home/apt|      Chris|June 22 2019|
|      5178| 79 dollars|Manhattan, Hell's...|Large Furnished R...|   private room|   Shunichi|June 24 2019|
|      5238|150 dollars|Manhattan, Chinatown|Cute & Cozy Lower...|Entire home/apt|        Ben|June 09 2019|
+----------+-----------+--------------------+--------------------+---------------+-----------+------------+
only showin

In [0]:
from pyspark.sql.functions import col, lower, regexp_replace, to_date

# Actualizamos las columnas y corregimos los problemas
airbnb_listings = (
    airbnb_listings
    .withColumn("price", regexp_replace(col("price"), " dollars", "").cast("int"))  # Eliminar " dollars" y convertir a entero
    .withColumn("room_type", lower(col("room_type")))  # Convertir a minúsculas
    .withColumn("last_review", to_date(col("last_review"), "MMMM dd yyyy"))  # Corregir nombre de columna y convertir a fecha
)

# Mostrar DataFrame actualizado
print("DataFrame actualizado:")
airbnb_listings.show(5)


DataFrame actualizado:
+----------+-----+--------------------+--------------------+---------------+-----------+-----------+
|listing_id|price|         nbhood_full|         description|      room_type|  host_name|last_review|
+----------+-----+--------------------+--------------------+---------------+-----------+-----------+
|      2595|  225|  Manhattan, Midtown|Skylit Midtown Ca...|entire home/apt|   Jennifer| 2019-05-21|
|      3831|   89|Brooklyn, Clinton...|Cozy Entire Floor...|entire home/apt|LisaRoxanne| 2019-07-05|
|      5099|  200|Manhattan, Murray...|Large Cozy 1 BR A...|entire home/apt|      Chris| 2019-06-22|
|      5178|   79|Manhattan, Hell's...|Large Furnished R...|   private room|   Shunichi| 2019-06-24|
|      5238|  150|Manhattan, Chinatown|Cute & Cozy Lower...|entire home/apt|        Ben| 2019-06-09|
+----------+-----+--------------------+--------------------+---------------+-----------+-----------+
only showing top 5 rows



* Fechas de las primeras y ultimas reseñas

In [0]:
from pyspark.sql.functions import col, lower, regexp_replace, to_date, min, max, avg
#obtenemos los valores minimos y maximos de las last_review
firts_reviewed = airbnb_listings.agg(min("last_review").alias("firts_reviewed")).collect()[0]["firts_reviewed"]
last_reviewed = airbnb_listings.agg(max("last_review").alias("last_reviewed")).collect()[0]["last_reviewed"]

print(f"Las reseñas mas antiguas y recientes fueron el {firts_reviewed} y  {last_reviewed}.")

Las reseñas mas antiguas y recientes fueron el 2019-01-01 y  2019-07-09.


* Count de habitaciones privadas

In [0]:
#filtramos las filas donde room_type es "private room" y con el count contamos el numero de filas 
nb_private_rooms = airbnb_listings.filter(col("room_type") == "private room").count()
print(f"De todas las ofertas, {bnb_private_rooms} son habitaciones privadas.")


De todas las ofertas, 1942 son habitaciones privadas.


* Precio promedio de los anuncios

In [0]:
#con avg calcular la media del precio y luego en el print con round redondeamos 2 decimales
avg_price = airbnb_listings.agg(avg("price").alias("avg_price")).collect()[0]["avg_price"]
print(f"El precio promedio de los anuncios es {round(avg_price, 2)} USD.")


El precio promedio de los anuncios es 143.28 USD.


* Crear un nuevo DataFrame con los resultados combinados

In [0]:

review_dates = spark.createDataFrame(
   [(firts_reviewed, last_reviewed, nb_private_rooms, round(avg_price, 2))],
   ["firts_reviewed", "last_reviewed", "nb_private_rooms", "avg_price"]
)
print("Nuevo DataFrame con resultados combinados:")
review_dates.show()

Nuevo DataFrame con resultados combinados:
+--------------+-------------+----------------+---------+
|firts_reviewed|last_reviewed|nb_private_rooms|avg_price|
+--------------+-------------+----------------+---------+
|    2019-01-01|   2019-07-09|            1942|   143.28|
+--------------+-------------+----------------+---------+

